# First load os and sys so I can update the sys.path with new functions

change the luminosity to 80 /fb

generate the 3 plots as in aewol paper.

In [1]:
import os
import sys

In [2]:
#take the paths to the functions we nedd
module_path = os.path.abspath(os.path.join('./pandasPlotting/'))
module2_path = os.path.abspath(os.path.join('./MlClasses/'))
module3_path = os.path.abspath(os.path.join('./MlFunctions/'))

In [3]:
# this part will include in the sys.path variables the paths for our new functions
if [module_path, module2_path, module3_path] not in sys.path:
    sys.path.append(module_path)

In [4]:
# here we are going to load what we will need, keras + tensorflow, plot functions, etc..
import matplotlib
matplotlib.use('Agg')
import pandas as pd
import numpy as np
import math
import time

from keras import callbacks

from pandasPlotting.Plotter import Plotter
from pandasPlotting.dfFunctions import expandArrays
from pandasPlotting.dtFunctions import featureImportance

from MlClasses.MlData import MlData
from MlClasses.Bdt import Bdt
from MlClasses.Dnn import Dnn
from MlClasses.ComparePerformances import ComparePerformances

from MlFunctions.DnnFunctions import significanceLoss,significanceLossInvert,significanceLoss2Invert ,significanceLossInvertSqrt,significanceFull,asimovSignificanceLoss,asimovSignificanceLossInvert,asimovSignificanceFull,truePositive,falsePositive

from linearAlgebraFunctions import gram,addGramToFlatDF

Using TensorFlow backend.


In [5]:
# I don't have patience to training 200 epochs ¯\_(ツ)_/¯
earlyStopping = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5)

In [6]:
#load our data files
signal=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/0L/feb/feb-0l-BSMlike/zh0p030lTCfeb.csv",sep='\s+',engine='python')
bkgd=pd.read_csv("../pyROOT_CPV_CPC/pp_wh/2dplots/analysis_with_cut/cpv_scan/charanjit_data/0L/feb/feb-0l-BSMlike/zhsm0lTCfeb.csv",sep='\s+',engine='python')


In [7]:
#combine them into one dataset
combined = pd.concat([signal,bkgd]).sample(frac=1)

In [8]:
print(combined.keys())

Index(['ptb1', 'ptb2', 'misset', 'pth', 'ptz', 'etah', 'phih', 'mtvh', 'ptvh',
       'dphib1met', 'signal'],
      dtype='object')


In [9]:
# change thes vars depend on which dataset you are loading, I will implement a better solution.
chosenVars = {
            # #A vanilla analysis with HL variables and lead 3 jets
            '0L':['ptb1', 'ptb2', 'misset', 'pth', 'ptz', 'etah', 'phih', 'mtvh', 'ptvh', 'dphib1met', 'signal'],
            #'2L':['ptb1', 'ptb2', 'ptl1', 'ptl2', 'pth', 'ptz', 'etah', 'phih',
            #     'deltarll', 'deltarbl', 'mtvh', 'ptvh', 'dphil1b1', 'dphil1b2', 'signal']
}

In [10]:
trainedModels={}

In [11]:
#needed to plot asimov significane
asimovSigLossSysts=[0.01,0.05,0.1,0.2,0.3,0.4,0.5]

In [12]:
# here I have included one archtecture I got from my ES scan, pls comment my entry and use dnn_batch4096 instead.
dnnConfigs={
    #'dnn_ZH_0L_cHW_0d001_batch_1024':{'epochs':200,'batch_size':1024,'dropOut':0.2,'l2Regularization':None,'hiddenLayers':[1.0],
    #             'optimizer':'adam', 'activation':'relu'}
    #'dnn_ZH_0L_cHW_0d01_batch_1024':{'epochs':200,'batch_size':1024,'dropOut':0.2,'l2Regularization':None,'hiddenLayers':[1.0],
    #             'optimizer':'adam', 'activation':'relu'}
    'dnn_ZH_0L_cHW_0d03_batch_1024':{'epochs':200,'batch_size':1024,'dropOut':0.2,'l2Regularization':None,'hiddenLayers':[1.0],
                 'optimizer':'adam', 'activation':'relu'}

}

In [13]:
#bsmlike signal cHW 0.03: 11.867fb, bg:0.89
#inclusive signal cHW 0.001: 2.198fb, bg:2.03fb
#inclusive signal cHW 0.01: 4.553fb, bg:2.03fb
#inclusive signal cHW 0.03: 14fb, bg:2.03fb

lumi=80. #luminosity in /fb
expectedSignal=11.867*lumi 
expectedBkgd=0.89*lumi #cross section of ttbar sample in fb times efficiency measured by Marco
systematic=0.5 #systematic for the asimov signficance

In [14]:
# I'm only running the DNN with binary cross entropy, letter I will include the other options.
# so far it running perfect, I'm testing with their custom loss function.
for varSetName,varSet in chosenVars.items():
    #Pick out the expanded arrays
    columnsInDataFrame = []
    for k in combined.keys():
        for v in varSet:
            #Little trick to ensure only the start of the string is checked
            if varSetName is '0L':
                if ' '+v+' ' in ' '+k+' ': columnsInDataFrame.append(k)
            elif ' '+v in ' '+k: columnsInDataFrame.append(k)


    #Select just the features we're interested in
    #For now setting NaNs to 0 for compatibility
    combinedToRun = combined[columnsInDataFrame].copy()
    combinedToRun.fillna(0,inplace=True)
    
    combinedToRun.index = np.arange(0,200000)
    mlData = MlData(combinedToRun,'signal')

    mlData.prepare(evalSize=0.0,testSize=0.3,limitSize=None)

    for name,config in dnnConfigs.items():
        dnn = Dnn(mlData,'testPlots/mlPlots/binary_crossentropy/bsmlike/'+varSetName+'/'+name)
        dnn.setup(hiddenLayers=config['hiddenLayers'], dropOut=config['dropOut'],
                  l2Regularization=config['l2Regularization'], optimizer=config['optimizer'],
                  activation=config['activation'],
                  loss='binary_crossentropy',
                extraMetrics=[
                    significanceLoss(expectedSignal,expectedBkgd),significanceFull(expectedSignal,expectedBkgd),
                    asimovSignificanceFull(expectedSignal,expectedBkgd,systematic),truePositive,falsePositive
                    ])
        dnn.fit(epochs=config['epochs'],batch_size=config['batch_size'])
        
        dnn.explainPredictions()
        dnn.diagnostics(batchSize=config['batch_size'])
        dnn.makeHepPlots(expectedSignal,expectedBkgd,asimovSigLossSysts,makeHistograms=False)

        trainedModels[varSetName+'_'+name]=dnn

/home/felipe/hepML/MlClasses/Dnn.py:101: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.history = self.model.fit(self.data.X_train.as_matrix(), self.data.y_train.as_matrix(), sample_weight=self.data.weights_train,
/home/felipe/hepML/MlClasses/Dnn.py:102: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  validation_data=(self.data.X_test.as_matrix(),self.data.y_test.as_matrix(),self.data.weights_test),


Train on 140000 samples, validate on 60000 samples
Epoch 1/200
140000/140000 [==============================] - 1s 7us/step - loss: 0.6501 - acc: 0.6162 - sigLoss: -486.6820 - significance: 22.9101 - asimovSignificance: 12.3342 - truePositive: 0.5790 - falsePositive: 0.3464 - val_loss: 0.5659 - val_acc: 0.7149 - val_sigLoss: -547.5568 - val_significance: 23.7150 - val_asimovSignificance: 17.0205 - val_truePositive: 0.6055 - val_falsePositive: 0.1765
Epoch 2/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5602 - acc: 0.7084 - sigLoss: -557.3877 - significance: 23.7617 - asimovSignificance: 16.4277 - truePositive: 0.6090 - falsePositive: 0.1917 - val_loss: 0.5413 - val_acc: 0.7278 - val_sigLoss: -572.5714 - val_significance: 23.5164 - val_asimovSignificance: 18.8226 - val_truePositive: 0.5929 - val_falsePositive: 0.1385
Epoch 3/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5486 - acc: 0.7182 - sigLoss: -571.1858 - significance: 23.6

140000/140000 [==============================] - 0s 3us/step - loss: 0.5367 - acc: 0.7321 - sigLoss: -580.1097 - significance: 24.3128 - asimovSignificance: 17.7367 - truePositive: 0.6354 - falsePositive: 0.1708 - val_loss: 0.5283 - val_acc: 0.7350 - val_sigLoss: -586.3993 - val_significance: 24.3969 - val_asimovSignificance: 17.8110 - val_truePositive: 0.6396 - val_falsePositive: 0.1705
Epoch 22/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5360 - acc: 0.7315 - sigLoss: -580.9814 - significance: 24.3151 - asimovSignificance: 17.6658 - truePositive: 0.6357 - falsePositive: 0.1724 - val_loss: 0.5279 - val_acc: 0.7349 - val_sigLoss: -584.8794 - val_significance: 24.3404 - val_asimovSignificance: 17.9122 - val_truePositive: 0.6365 - val_falsePositive: 0.1675
Epoch 23/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5355 - acc: 0.7318 - sigLoss: -581.7597 - significance: 24.3565 - asimovSignificance: 17.6226 - truePositive: 0.6378 - fa

140000/140000 [==============================] - 0s 3us/step - loss: 0.5359 - acc: 0.7316 - sigLoss: -580.2096 - significance: 24.3467 - asimovSignificance: 17.6214 - truePositive: 0.6373 - falsePositive: 0.1737 - val_loss: 0.5278 - val_acc: 0.7348 - val_sigLoss: -584.1788 - val_significance: 24.4581 - val_asimovSignificance: 17.6706 - val_truePositive: 0.6431 - val_falsePositive: 0.1744
Epoch 42/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5354 - acc: 0.7314 - sigLoss: -580.6136 - significance: 24.3838 - asimovSignificance: 17.5268 - truePositive: 0.6394 - falsePositive: 0.1764 - val_loss: 0.5280 - val_acc: 0.7346 - val_sigLoss: -584.6838 - val_significance: 24.4439 - val_asimovSignificance: 17.6760 - val_truePositive: 0.6423 - val_falsePositive: 0.1740
Epoch 43/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5354 - acc: 0.7315 - sigLoss: -580.7045 - significance: 24.3568 - asimovSignificance: 17.5776 - truePositive: 0.6380 - fa

140000/140000 [==============================] - 0s 3us/step - loss: 0.5355 - acc: 0.7317 - sigLoss: -580.7667 - significance: 24.3976 - asimovSignificance: 17.5357 - truePositive: 0.6401 - falsePositive: 0.1764 - val_loss: 0.5280 - val_acc: 0.7350 - val_sigLoss: -583.1169 - val_significance: 24.4951 - val_asimovSignificance: 17.6260 - val_truePositive: 0.6451 - val_falsePositive: 0.1759
Epoch 62/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5354 - acc: 0.7318 - sigLoss: -580.3043 - significance: 24.3686 - asimovSignificance: 17.6021 - truePositive: 0.6385 - falsePositive: 0.1744 - val_loss: 0.5280 - val_acc: 0.7351 - val_sigLoss: -582.1262 - val_significance: 24.4834 - val_asimovSignificance: 17.6637 - val_truePositive: 0.6444 - val_falsePositive: 0.1750
Epoch 63/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5356 - acc: 0.7322 - sigLoss: -580.4675 - significance: 24.3916 - asimovSignificance: 17.5947 - truePositive: 0.6397 - fa

...

140000/140000 [==============================] - 0s 3us/step - loss: 0.5356 - acc: 0.7318 - sigLoss: -580.6595 - significance: 24.3876 - asimovSignificance: 17.5782 - truePositive: 0.6395 - falsePositive: 0.1754 - val_loss: 0.5278 - val_acc: 0.7349 - val_sigLoss: -583.0069 - val_significance: 24.4256 - val_asimovSignificance: 17.7478 - val_truePositive: 0.6413 - val_falsePositive: 0.1723
Epoch 162/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5356 - acc: 0.7321 - sigLoss: -580.2668 - significance: 24.3924 - asimovSignificance: 17.5911 - truePositive: 0.6398 - falsePositive: 0.1754 - val_loss: 0.5279 - val_acc: 0.7346 - val_sigLoss: -585.5370 - val_significance: 24.4474 - val_asimovSignificance: 17.6784 - val_truePositive: 0.6425 - val_falsePositive: 0.1741
Epoch 163/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5349 - acc: 0.7320 - sigLoss: -580.8459 - significance: 24.3696 - asimovSignificance: 17.6146 - truePositive: 0.6385 - 

140000/140000 [==============================] - 0s 3us/step - loss: 0.5357 - acc: 0.7321 - sigLoss: -581.2424 - significance: 24.4018 - asimovSignificance: 17.5682 - truePositive: 0.6403 - falsePositive: 0.1758 - val_loss: 0.5281 - val_acc: 0.7348 - val_sigLoss: -585.0197 - val_significance: 24.4534 - val_asimovSignificance: 17.6869 - val_truePositive: 0.6428 - val_falsePositive: 0.1740
Epoch 182/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5353 - acc: 0.7320 - sigLoss: -580.1146 - significance: 24.3431 - asimovSignificance: 17.6599 - truePositive: 0.6371 - falsePositive: 0.1728 - val_loss: 0.5280 - val_acc: 0.7345 - val_sigLoss: -588.1526 - val_significance: 24.5419 - val_asimovSignificance: 17.5015 - val_truePositive: 0.6478 - val_falsePositive: 0.1795
Epoch 183/200
140000/140000 [==============================] - 0s 3us/step - loss: 0.5352 - acc: 0.7320 - sigLoss: -581.3681 - significance: 24.4097 - asimovSignificance: 17.5505 - truePositive: 0.6407 - 

/home/felipe/hepML/MlClasses/Dnn.py:221: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  report = self.model.evaluate(X_test.as_matrix(), y_test.as_matrix(), sample_weight=weights_test, batch_size=batchSize)
/home/felipe/hepML/MlClasses/Dnn.py:227: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  classificationReport(self.model.predict_classes(X_test.as_matrix()),self.model.predict(X_test.as_matrix()),y_test,f)


140000/140000 [==============================] - 0s 1us/step


/home/felipe/hepML/MlClasses/Dnn.py:232: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  report = self.model.evaluate(X_train.as_matrix(), y_train.as_matrix(), sample_weight=weights_train, batch_size=batchSize)
/home/felipe/hepML/MlClasses/Dnn.py:236: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  classificationReport(self.model.predict_classes(X_train.as_matrix()),self.model.predict(X_train.as_matrix()),y_train,f)
/home/felipe/hepML/MlClasses/Dnn.py:254: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  rocCurve(self.model.predict(self.data.X_test.as_matrix()), self.data.y_test,self.output)
/home/felipe/hepML/MlClasses/Dnn.py:255: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  rocCurve(self.model.predict(self.data.X_train.as_matrix()),self.data.y_train,self.output,append='_train')
/home/felipe/hepML/MlClasse